##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Word Embeddings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This tutorial shows how to train a sentiment classifer on the IMDB dataset using learned word embeddings. As a bonus, we show how to visualize these embeddings in the [TensorFlow Embedding Projector](http://projector.tensorflow.org). 

First, here's a bit of background. Before we can build a model to predict the sentiment of a review, first we will need a way to represent the words of the review as numbers, so they can be processed by our network. There are several strategies to convert words to numbers.

As a first attempt, we might one-hot encode each word. One problem with this approach is efficiency. A one-hot encoded vector is sparse (meaning, most indicices are zero). Imagine we have 10,000 words in our vocabulary. To one-hot encode each one, we would create a vector where 99.99% of the elements are zero!

Instead, we can encode each word using a unique number. For example, we might assign 1 to 'the', 42 to 'dog', and 96 to 'cat', and so on. Using these numbers, we could encode a sentence like "The dog and cat sat on the mat" as \[1, 42, 96, ...\]. One problem still remains. Although we know dogs and cats are related, our representation doesn't encode that information for the classifier (the numbers 42 and 96 were arbitrarily chosen). 

Unlike the above methods, a word embedding is learned from data. An embedding represents each word as a n-dimensional vector of floating point values. These values are traininable parameters, weights learned while training the model. After training, we hope that similar words will be close together in the embedding space. We can visualize the learned embeddings by projecting them down to a 2- or 3-dimensional space.

There are two ways to obtain word embeddings:

* Learn word embeddings jointly with the main task you care about (e.g. sentiment classification). In this case, you would start with random word vectors, then learn your word vectors in the same way that you learn the weights of a neural network.

* Load word embeddings into your model that were pre-computed using a different machine learning task than the one you are trying to solve. These are called "pre-trained word embeddings".

Here, we will take the first approach.

In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

# Download the IMDB dataset

The IMDB dataset comes packaged with TensorFlow. It has already been preprocessed such that the reviews (sequences of words) have been converted to sequences of integers, where each integer represents a specific word in a dictionary.

In [ ]:
imdb = keras.datasets.imdb

# Number of words to consider as features
num_words = 1000

# load IMDB dataset as lists of integers
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)

The argument num_words=1000 keeps the top 1,000 most frequently occurring words in the training data.

In [ ]:
print("Training examples: {}, labels: {}".format(len(train_data), len(train_labels)))

The text of reviews have been converted to integers, where each integer represents a specific word in a dictionary. Here's what the first review looks like:

In [ ]:
print(train_data[0])

Movie reviews may be different lengths. The below code shows the number of words in the first and second reviews. Since inputs to a neural network must be the same length, we'll need to resolve this.

In [ ]:
len(train_data[0]), len(train_data[1])

We will pad the arrays so they all have the same length, using the [https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences](pad_sequences) method. In this case, TensorFlow will create new matrix of shape ```max_len * num_examples```:

In [ ]:
# Cut texts after this number of words 
max_len = 250

# Convert our lists of integers into 2D tensors
train_data = keras.preprocessing.sequence.pad_sequences(train_data, 
                                                        maxlen=max_len)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, 
                                                       maxlen=max_len)

print(train_data.shape)

Notice the pad sequences method worked by prepending '0's to the start of the sequence:

In [ ]:
print(train_data[0])

We are now ready to build our model. We will use an [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer to map from an integer that corresponds to a word, to a vector of floating point weights (the embedding). These weights are learned when we train the model.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D

embedding_dimension = 16

model = Sequential()
model.add(Embedding(num_words, embedding_dimension, input_length=max_len))

# Our output is a 3D tensor of shape (samples, vocab_size, embedding_dimension)
# we will use `GlobalAveragePooling` it into a 2D tensor of shape (samples, vocab_size * embedding_dimension)
model.add(GlobalAveragePooling1D())

# Add a classifier on top.
model.add(Dense(1, activation='sigmoid') )
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Our classifier has a validation accuracy of about 83%. Note that we make use of only the first 250 words in each review. We are also flattening our embedding and passing it to a single Dense layer, which treats each word separately without taking into consideration the ordering of the words in the sequence. To reach higher accuracy, it would be helpful use a recurrent layer or 1D convolution which will take the sequence of the words into consideration.

# Visualize Embeddings with the Embedding Projector

Recall the reviews are encoded as series of integers in our training data. Before we can visualize the learned embeddings, first we will need to determine which word corresponds to each number. In this case, the IMDB dataset includes a utility method ```.word_index()``` that contains a mapping from words to numbers. We will use this to build a reversed word index, which maps from numbers to words.

In [ ]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index[i] for i in text])

Now we can use the decode_review function to display the text for the first review. You will see padding at the beginning, since this review was shorter than our 250 word maximum length.

In [ ]:
decode_review(train_data[0])

Now that we have the number to word mapping, we are ready to retrieve the learned embedding from the model. This gives us a matrix of weights. Each row corresponds to the embedding for that number in our ```reversed_word_dict``` above, and the corresponding word can be found in ```word_index```.

We retrieve the weights by using the ```model.layers``` and ```model.weights``` methods. In this case, the embedding layer is the first layer we added to the model.



In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # 1000, 16. Each word is mapped to an embedding vector.

Next, we will format these for visualization in the embedding projector. To do so, we will need to provide two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).

In [ ]:
out_v = open('vecs.tsv', 'w')
out_m = open('meta.tsv', 'w')
for word_num in range(num_words):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

If you are running this tutorial in [Colaboratory](https://colab.research.google.com), you can use the following snippet to download these files to your local machine.

In [ ]:
from google.colab import files
files.download('vecs.tsv')
files.download('meta.tsv')

Now, you can open the [Embedding Projector](http://projector.tensorflow.org/) in a new window, and click on 'Load data'. Upload the ```vecs.tsv``` and ```meta.tsv``` files from above. Next, click 'Search', and type in a word to find its closest neighbors. With this small dataset, not all of the learned embeddings will be interpretable, though some will be! 

For example, try searching for 'beautiful'. The learned embeddings you see may be different, they depend on random weight initialization used by the model. When the author of this tutorial ran it, they saw "loved" and "wonderful" were the closest neighbors. Likewise, the closest neigbhors for "lame" were "awful, and poorly".

# Next steps
* To learn more about Word Embeddings, we recommend browing [this](https://www.tensorflow.org/tutorials/representation/word2vec) older tutorial (the code is out of date, and we recommend skipping it in favor of the newer version here, but the explanation and diagrams are useful).

* [TensorFlow Hub](https://www.tensorflow.org/hub/) contains large databases of pretrained word embeddings you can download and reuse in your projects (although at the time of writing, these use a different progamming style than the one in this tutorial).